In [1]:
import scvelo as scv
import pandas as pd
import numpy as np
import scanpy as sc
import glob

In [2]:
sampleL = ["ncc", "mes"]
sample = 'ncc_mes_eu'
results_file = '/home/linxy29/holab/linxy/iPSC/veloAE/' + sample + '_seurate.h5ad'  # the file that will store the analysis results

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [4]:
# get meta-data information
path = "/home/linxy29/holab/linxy/iPSC/veloAE/"
cellID_obs = pd.read_csv(path + sample + '_cellID.csv')
print(cellID_obs.head())
print(len(cellID_obs))
umap_cord = pd.read_csv(path + sample + '_embeddings.csv')
print(umap_cord)
cell_clusters = pd.read_csv(path + sample + '_clusters.csv')

                      x
0  AAACCCATCAGCAATC-1_1
1  AAACGAACACGGTGCT-1_1
2  AAACGAACATTGTCGA-1_1
3  AAACGCTCAAGAGGTC-1_1
4  AAACGCTGTGCCCGTA-1_1
22522
                 Unnamed: 0    UMAP_1    UMAP_2
0      AAACCCATCAGCAATC-1_1  0.761892  5.784208
1      AAACGAACACGGTGCT-1_1 -2.664008  2.284072
2      AAACGAACATTGTCGA-1_1 -0.469658  8.824039
3      AAACGCTCAAGAGGTC-1_1 -1.168397  8.088729
4      AAACGCTGTGCCCGTA-1_1  3.131986  2.131230
...                     ...       ...       ...
22517  TTTGTTGGTAGACGTG-1_3 -0.302517 -5.987935
22518  TTTGTTGGTGAGCGAT-1_3  2.782733 -0.404670
22519  TTTGTTGGTTCTCTAT-1_3  3.069862 -1.504854
22520  TTTGTTGTCACCTCAC-1_3  0.649504 -9.676390
22521  TTTGTTGTCGGATAAA-1_3  3.593241 -8.271546

[22522 rows x 3 columns]


In [5]:
filenameL = []
os.chdir("/home/linxy29/holab/linxy/iPSC/day7_velocyto/")
for file in glob.glob("*.loom"):
    filenameL.append(file)
print(filenameL)

['ThermoFiPSC_D7NOTO_drug_sample_alignments_PUHUH.loom', '190PBMCiPSC_D7NOTO_sample_alignments_OV87U.loom', 'ThermoFiPSC_D7NOTO_sample_alignments_BT23H.loom', '190PBMCiPSC_D7NT_sample_alignments_LNJIY.loom', 'ThermoFiPSC_D7NT_sample_alignments_Z46BN.loom', '190PBMCiPSC_D7NOTO_drug_sample_alignments_P1WVY.loom']


In [6]:
eudata =  scv.read("/home/linxy29/holab/linxy/iPSC/day7_velocyto/" + filenameL[0], cache=True)
eudata.var_names_make_unique()

... reading from cache file cache/home-linxy29-holab-linxy-iPSC-day7_velocyto-ThermoFiPSC_D7NOTO_drug_sample_alignments_PUHUH.h5ad


In [7]:
for sample in filenameL[1:]:
    tempdata = scv.read("/home/linxy29/holab/linxy/iPSC/day7_velocyto/" + sample, cache=True)
    tempdata.var_names_make_unique()
    eudata = eudata.concatenate(tempdata, join='outer', index_unique=None)
print(eudata)
obs_name_temp = eudata.obs_names.str.split(":")
eudata.obs_names = [item[1] + '_3' for item in obs_name_temp]
print(eudata.obs_names)

... reading from cache file cache/home-linxy29-holab-linxy-iPSC-day7_velocyto-190PBMCiPSC_D7NOTO_sample_alignments_OV87U.h5ad
... reading from cache file cache/home-linxy29-holab-linxy-iPSC-day7_velocyto-ThermoFiPSC_D7NOTO_sample_alignments_BT23H.h5ad
... reading from cache file cache/home-linxy29-holab-linxy-iPSC-day7_velocyto-190PBMCiPSC_D7NT_sample_alignments_LNJIY.h5ad
... reading from cache file cache/home-linxy29-holab-linxy-iPSC-day7_velocyto-ThermoFiPSC_D7NT_sample_alignments_Z46BN.h5ad
... reading from cache file cache/home-linxy29-holab-linxy-iPSC-day7_velocyto-190PBMCiPSC_D7NOTO_drug_sample_alignments_P1WVY.h5ad
AnnData object with n_obs × n_vars = 10656 × 61544
    obs: 'batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['AAGCGAGTCCCGAATA_3', 'AAATGGATCTGCGGGT_3', 'AACGTCACATGATGCT_3',
       'AAGAACAAGTCCTGCG_3', 'AATTTCCCAGTCGCTG_3', 'AACCTTTAGCAGTACG_3',
       'AAGAACAGTCGGATTT_3', 'ACATGC

In [8]:
## add ncc data
ldata =  scv.read('/home/linxy29/holab/linxy/vivian/' + sampleL[0] + '_cellranger/velocyto/' + sampleL[0] + '_cellranger.loom', cache=True)
obs_name_rep = ldata.obs_names.str.replace('ncc_cellranger:|x', '') + '_1'
ldata.obs_names = obs_name_rep
ldata.var_names_make_unique()
#adata.obs_names = sampleL[0] + '_' + adata.obs_names
print(ldata)
print(ldata.obs_names)

... reading from cache file cache/home-linxy29-holab-linxy-vivian-ncc_cellranger-velocyto-ncc_cellranger.h5ad
AnnData object with n_obs × n_vars = 2631 × 36601
    obs: 'Clusters', '_X', '_Y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['AAAGTCCTCTTAAGGC_1', 'AACAAAGCAGCTCTGG_1', 'AAAGGTAGTCATCCCT_1',
       'AACCCAATCCGTGACG_1', 'AACCATGAGATGGTCG_1', 'AAGATAGGTCATTGCA_1',
       'AAGGAATCATAGATCC_1', 'AACCATGTCGCACGAC_1', 'AAGCATCTCGTAATGC_1',
       'AACAAAGAGTGAGCCA_1',
       ...
       'TTTGACTAGGCACCAA_1', 'TTTCGATTCGAGCCTG_1', 'TTTGACTTCAAGTGTC_1',
       'TTTGTTGAGCATGATA_1', 'TTTCGATAGTGTCATC_1', 'TTTATGCTCTCCATAT_1',
       'TTTATGCAGCTACTGT_1', 'TTTGACTGTGCCCAGT_1', 'TTTCGATTCTTTCCAA_1',
       'TTTCGATTCCGTGACG_1'],
      dtype='object', name='CellID', length=2631)


In [9]:
## add mes data
templdata = scv.read('/home/linxy29/holab/linxy/vivian/' + sampleL[1] + '_cellranger/velocyto/' + sampleL[1] + '_cellranger.loom', cache=True)
obs_name_rep = templdata.obs_names.str.replace('mes_cellranger:|x', '') + '_2'
templdata.obs_names = obs_name_rep
templdata.var_names_make_unique()
print(templdata)
print(templdata.obs_names)
ldata = ldata.concatenate(templdata, join='outer', index_unique=None)
print(ldata)

... reading from cache file cache/home-linxy29-holab-linxy-vivian-mes_cellranger-velocyto-mes_cellranger.h5ad
AnnData object with n_obs × n_vars = 10804 × 36601
    obs: 'Clusters', '_X', '_Y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['AAAGTCCAGAAGTCTA_2', 'AAAGTCCAGGGCTTCC_2', 'AAACGCTCATCTCGTC_2',
       'AAAGTGAAGTAGGCCA_2', 'AAACGCTAGCTGACAG_2', 'AAACGAATCGGTCTGG_2',
       'AAAGTGATCTCTTGCG_2', 'AAAGGTAGTGAACGGT_2', 'AAAGGTAGTTACTCAG_2',
       'AAAGTGAGTGTGAGCA_2',
       ...
       'TTTGGTTCAACCGACC_2', 'TTTGATCTCATGGTAC_2', 'TTTCCTCGTTGCTAGT_2',
       'TTTGACTGTCTAGTGT_2', 'TTTCCTCTCAACTCTT_2', 'TTTGTTGTCATCTGTT_2',
       'TTTGTTGTCGAGCTGC_2', 'TTTGTTGTCTCATGGA_2', 'TTTGTTGTCCTGGGAC_2',
       'TTTGTTGTCCCAAGTA_2'],
      dtype='object', name='CellID', length=10804)
AnnData object with n_obs × n_vars = 13435 × 36601
    obs: 'Clusters', '_X', '_Y', 'batch'
    var: 'Accession', 'Chromosome', '

In [10]:
ldata = ldata.concatenate(eudata, join='outer', index_unique=None)
print(ldata)

AnnData object with n_obs × n_vars = 24091 × 74559
    obs: 'Clusters', '_X', '_Y', 'batch'
    var: 'Accession-0', 'Chromosome-0', 'End-0', 'Start-0', 'Strand-0', 'Accession-1', 'Chromosome-1', 'End-1', 'Start-1', 'Strand-1'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


## barcodes

In [11]:
## change obs_names
print(ldata.obs_names)
print(np.unique(ldata.obs_names).size == len(ldata.obs_names))
print(len(ldata.obs_names))

Index(['AAAGTCCTCTTAAGGC_1', 'AACAAAGCAGCTCTGG_1', 'AAAGGTAGTCATCCCT_1',
       'AACCCAATCCGTGACG_1', 'AACCATGAGATGGTCG_1', 'AAGATAGGTCATTGCA_1',
       'AAGGAATCATAGATCC_1', 'AACCATGTCGCACGAC_1', 'AAGCATCTCGTAATGC_1',
       'AACAAAGAGTGAGCCA_1',
       ...
       'TTGGATGTCAACGTGT_3', 'TTAGGGTGTCAGTCGC_3', 'TTTCGATCAAGACGGT_3',
       'TTTGGTTAGTGAGCCA_3', 'TTTATGCTCAGGACGA_3', 'TTTGATCAGTGGCCTC_3',
       'TTTGTTGGTAGACGTG_3', 'TTTCGATTCCTGGGAC_3', 'TTTCGATAGTAGGATT_3',
       'TTTGGAGAGGTAACTA_3'],
      dtype='object', length=24091)
True
24091


In [12]:
## change cellID name in metadata
cellID_obs['x'] = cellID_obs['x'].str.replace('-1', '')
print(np.unique(cellID_obs['x']).size == len(cellID_obs['x']))  ## check whether have duplicate barcodes
print(len(cellID_obs))
print(cellID_obs)

True
22522
                        x
0      AAACCCATCAGCAATC_1
1      AAACGAACACGGTGCT_1
2      AAACGAACATTGTCGA_1
3      AAACGCTCAAGAGGTC_1
4      AAACGCTGTGCCCGTA_1
...                   ...
22517  TTTGTTGGTAGACGTG_3
22518  TTTGTTGGTGAGCGAT_3
22519  TTTGTTGGTTCTCTAT_3
22520  TTTGTTGTCACCTCAC_3
22521  TTTGTTGTCGGATAAA_3

[22522 rows x 1 columns]


In [14]:
filtered_barcode = np.intersect1d(cellID_obs['x'],ldata.obs_names)
len(filtered_barcode)

22024

In [15]:
filtered_ldata = ldata[filtered_barcode].copy()
print(filtered_ldata)

AnnData object with n_obs × n_vars = 22024 × 74559
    obs: 'Clusters', '_X', '_Y', 'batch'
    var: 'Accession-0', 'Chromosome-0', 'End-0', 'Start-0', 'Strand-0', 'Accession-1', 'Chromosome-1', 'End-1', 'Start-1', 'Strand-1'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


Add UMAP and annotation

In [16]:
ldata_index = pd.DataFrame(filtered_ldata.obs.index)
ldata_index.columns = ['CellID']
print(ldata_index)

                   CellID
0      AAACCCAAGACATCAA_2
1      AAACCCAAGACGCATG_2
2      AAACCCAAGATGTTGA_2
3      AAACCCAAGCCGATCC_3
4      AAACCCAAGGTCATCT_2
...                   ...
22019  TTTGTTGTCATTGCGA_2
22020  TTTGTTGTCCCAAGTA_2
22021  TTTGTTGTCCTGGGAC_2
22022  TTTGTTGTCGAGCTGC_2
22023  TTTGTTGTCTCATGGA_2

[22024 rows x 1 columns]


In [17]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'CellID'})
#umap_cord = umap_cord.rename(columns = {'Cell ID':'CellID'})
umap_cord['CellID'] = umap_cord['CellID'].str.replace('-1', '')
umap_ordered = ldata_index.merge(umap_cord, on = "CellID")
print(umap_cord)

                   CellID    UMAP_1    UMAP_2
0      AAACCCATCAGCAATC_1  0.761892  5.784208
1      AAACGAACACGGTGCT_1 -2.664008  2.284072
2      AAACGAACATTGTCGA_1 -0.469658  8.824039
3      AAACGCTCAAGAGGTC_1 -1.168397  8.088729
4      AAACGCTGTGCCCGTA_1  3.131986  2.131230
...                   ...       ...       ...
22517  TTTGTTGGTAGACGTG_3 -0.302517 -5.987935
22518  TTTGTTGGTGAGCGAT_3  2.782733 -0.404670
22519  TTTGTTGGTTCTCTAT_3  3.069862 -1.504854
22520  TTTGTTGTCACCTCAC_3  0.649504 -9.676390
22521  TTTGTTGTCGGATAAA_3  3.593241 -8.271546

[22522 rows x 3 columns]


In [18]:
umap_ordered = umap_ordered.iloc[:,1:]
filtered_ldata.obsm['X_umap'] = umap_ordered.values

In [19]:
cell_clusters = cell_clusters.rename(columns = {'Unnamed: 0':'CellID'})
cell_clusters['CellID'] = cell_clusters['CellID'].str.replace('-1', '')
cell_clusters = ldata_index.merge(cell_clusters, on = "CellID")
print(cell_clusters)

                   CellID   x
0      AAACCCAAGACATCAA_2   1
1      AAACCCAAGACGCATG_2   0
2      AAACCCAAGATGTTGA_2   3
3      AAACCCAAGCCGATCC_3  17
4      AAACCCAAGGTCATCT_2   3
...                   ...  ..
22019  TTTGTTGTCATTGCGA_2  13
22020  TTTGTTGTCCCAAGTA_2  10
22021  TTTGTTGTCCTGGGAC_2   1
22022  TTTGTTGTCGAGCTGC_2   9
22023  TTTGTTGTCTCATGGA_2   1

[22024 rows x 2 columns]


In [20]:
cell_clusters = cell_clusters.iloc[:,1:]
filtered_ldata.obs['seurat_clusters'] = cell_clusters.values
print(filtered_ldata)

AnnData object with n_obs × n_vars = 22024 × 74559
    obs: 'Clusters', '_X', '_Y', 'batch', 'seurat_clusters'
    var: 'Accession-0', 'Chromosome-0', 'End-0', 'Start-0', 'Strand-0', 'Accession-1', 'Chromosome-1', 'End-1', 'Start-1', 'Strand-1'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


In [23]:
filtered_ldata.write(results_file)